In [1]:
from __future__ import division, print_function
%matplotlib inline
import utils;
from importlib import reload; reload(utils)
from utils import *

Using Theano backend.


In [2]:
path = "../../dataset/ml-latest-small/"
model_path = path
if not os.path.exists(model_path): os.mkdir(model_path)
batch_size=64

In [3]:
ratings = pd.read_csv(path+'ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
len(ratings)

100836

创建电影id到电影名称的字典，方便显示使用

In [5]:
movie_names = pd.read_csv(path+'movies.csv').set_index('movieId')['title'].to_dict()

In [6]:
users = ratings.userId.unique()
movies = ratings.movieId.unique()

创建用户id到索引的字典，创建用户id到索引的字典

In [7]:
userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}

In [8]:
type(userid2idx)

dict

In [9]:
type(ratings.userId)

pandas.core.series.Series

把电影id和用户id变成连续的整数，这样才能使用embeddings层

In [10]:
ratings.movieId = ratings.movieId.apply(lambda x: movieid2idx[x])
ratings.userId = ratings.userId.apply(lambda x: userid2idx[x])

In [11]:
user_min, user_max, movie_min, movie_max = (ratings.userId.min(), 
    ratings.userId.max(), ratings.movieId.min(), ratings.movieId.max())
user_min, user_max, movie_min, movie_max

(0, 609, 0, 9723)

In [12]:
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()
n_users, n_movies

(610, 9724)

embedding层中潜在因子的数量

In [13]:
n_factors = 50

seed随机种子，相同的种子每次的随机值相同

In [14]:
np.random.seed = 42

In [15]:
msk = np.random.rand(len(ratings)) < 0.8

In [16]:
msk

array([ True,  True,  True, ...,  True,  True,  True])

In [17]:
trn = ratings[msk]
val = ratings[~msk]

In [18]:
len(trn)

80541

In [19]:
len(val)

20295

## 创建交叉表

使用最流行的电影和最爱看电影的用户创建一张交叉表，这不是创建模型前必须的，仅仅是一个例子。

In [20]:
g=ratings.groupby('userId')['rating'].count()
topUsers=g.sort_values(ascending=False)[:15]

In [57]:
topUsers

userId
413    2698
598    2478
473    2108
447    1864
273    1346
609    1302
67     1260
379    1218
605    1115
287    1055
248    1046
386    1027
181     977
306     975
602     943
Name: rating, dtype: int64

In [58]:
g[:15]

userId
0     232
1      29
2      39
3     216
4      44
5     314
6     152
7      47
8      46
9     140
10     64
11     32
12     31
13     48
14    135
Name: rating, dtype: int64

In [59]:
ratings[:15]

,userId,movieId,rating,timestamp
0,0,0,4.0,964982703
1,0,1,4.0,964981247
2,0,2,4.0,964982224
3,0,3,5.0,964983815
4,0,4,5.0,964982931
5,0,5,3.0,964982400
6,0,6,5.0,964980868
7,0,7,4.0,964982176
8,0,8,5.0,964984041
9,0,9,5.0,964984100


In [60]:
g=ratings.groupby('movieId')['rating'].count()
topMovies=g.sort_values(ascending=False)[:15]

In [61]:
top_r = ratings.join(topUsers, rsuffix='_r', how='inner', on='userId')

In [62]:
top_r = top_r.join(topMovies, rsuffix='_r', how='inner', on='movieId')

In [63]:
pd.crosstab(top_r.userId, top_r.movieId, top_r.rating, aggfunc=np.sum)

movieId,0,4,7,15,16,20,26,28,34,68,166,184,192,232,478
userId,,,,,,,,,,,,,,,
67,2.5,3.0,2.5,5.0,2.0,3.5,3.5,4.0,3.5,5.0,4.5,5.0,2.5,3.0,3.5
181,4.0,4.5,3.5,3.5,5.0,5.0,3.5,4.0,4.5,3.0,5.0,5.0,5.0,4.5,2.0
248,4.0,4.0,5.0,5.0,4.0,4.5,4.0,4.5,4.0,5.0,5.0,4.5,5.0,4.5,4.0
273,4.0,4.0,4.5,3.0,5.0,4.5,3.5,4.0,4.0,4.5,4.0,5.0,5.0,4.5,4.5
287,4.5,NaN,5.0,5.0,5.0,5.0,2.0,5.0,5.0,4.5,3.0,NaN,3.5,5.0,4.0
306,4.0,4.5,3.5,3.5,4.5,4.0,3.5,4.5,4.5,3.0,3.5,4.0,4.0,4.5,2.5
379,5.0,4.0,4.0,5.0,5.0,5.0,5.0,NaN,5.0,5.0,4.5,NaN,4.0,3.0,5.0
386,NaN,4.5,3.5,4.5,5.0,4.0,3.0,NaN,4.0,4.5,4.0,4.5,4.5,3.5,3.5
413,4.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0


## 点积

最基础的模型是电影embedding层和用户embedding层的点积

In [67]:
user_in = Input(shape=(1,), dtype='int64', name='user_in')
u = Embedding(n_users, n_factors, input_length=1, W_regularizer=l2(1e-4))(user_in)
movie_in = Input(shape=(1,), dtype='int64', name='movie_in')
m = Embedding(n_movies, n_factors, input_length=1, W_regularizer=l2(1e-4))(movie_in)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(610, 50, input_length=1, embeddings_regularizer=<keras.reg...)`
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(9724, 50, input_length=1, embeddings_regularizer=<keras.reg...)`
  after removing the cwd from sys.path.


In [69]:
x = merge([u, m], mode='dot')

TypeError: 'module' object is not callable

In [ ]:
x = Flatten()(x)
model = Model([user_in, movie_in], x)
model.compile(Adam(0.001), loss='mse')